In [1]:

import numpy as np
import pandas as pd
import time
import tqdm 

from scipy.spatial.distance import cosine

In [2]:
from nltk.corpus import stopwords
# ...  
#filtered_words = list(filter(lambda word: word not in stopwords.words('english'), word_list))
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zaiem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import gensim 
from gensim.models import Word2Vec

C:\Anaconda3\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
with open("../src_medium_train.txt", encoding="utf8") as f:
    src_questions = f.readlines()
print(src_questions[-3:-1])
src_questions = [i[0:-2] for i in src_questions]
with open("../tgt_medium_train.txt",encoding="utf8" ) as f:
    tgt_questions = f.readlines()
tgt_questions = [i[0:-2] for i in tgt_questions]
all_questions  = src_questions + tgt_questions
print(len(tgt_questions))
print(len(src_questions))
src_questions[0:10]

['How many calories are in a taco from taco villa ?\n', 'What is italy s primary exports ?\n']
300000
300000


['How can you shop online without a credit card in canada ',
 'Most popular X men ',
 'Akbar s 9 ratna ',
 'Where is oil filter on 1994 protoge ',
 'How do you beat the pink thing in pirate islant in poptropica ',
 'What is the french name for dollar bills ',
 'What are Molecules that are always moving randomly ',
 'Are dolphins carnivore or ominvore ',
 'Where was an apple found ',
 'How many proteins in chick peas ']

In [6]:
with open("./src-test.txt", encoding="utf8") as f:
    src_test = f.readlines()
src_test = [i[0:-2] for i in src_test]

In [4]:
elements = pd.read_csv("working.csv")
elements.head()

,Unnamed: 0,description,narrative,title
0,0,Identify positive accomplishments of the Hubbl...,Documents are relevant that show the Hubble te...,Hubble Telescope Achievements
1,1,Identify hydroelectric projects proposed or un...,Relevant documents would contain as a minimum ...,New Hydroelectric Projects
2,2,Evidence that radio waves from radio towers or...,Persons living near radio towers and more rece...,Radio Waves and Brain Cancer
3,3,Commercial harvesting of marine vegetation suc...,Recent research has shown that marine vegetati...,Marine Vegetation
4,4,Isolate instances of fraud or embezzlement in ...,A relevant document is any report that identif...,International Art Crime


In [5]:
descriptions = elements["description"].tolist()

In [25]:
with open("queries.txt", encoding="utf8", mode="w") as q : 
    for i in descriptions : 
        q.write(i)
        q.write("\n")

In [26]:
titles = elements["title"].tolist()
with open("titles.txt", encoding="utf8", mode="w") as q : 
    for i in titles : 
        q.write(i)
        q.write("\n")

In [6]:
from nltk import word_tokenize

In [7]:
tokenized_all_questions = [word_tokenize(descriptions[i]) for i in tqdm.tqdm(range(len(descriptions)))]

100%|████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 2168.25it/s]


In [18]:
import fasttext
model = fasttext.load_model('GoogleNews-vectors-negative300.bin')


Exception: fastText: Cannot load GoogleNews-vectors-negative300.bin due to C++ extension failed to allocate the memory

In [8]:
%%time
from gensim.models import KeyedVectors


Wall time: 0 ns


In [9]:
%%time
word_vectors = KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True) 

Wall time: 44.7 s


In [10]:
t = word_vectors.get_vector("world")


In [11]:
query= "what is the biggest country in Africa"

In [12]:
def get_query_cent(query): 
    tokens = word_tokenize(query)
    centroid = [0] * 300
    for i in tokens : 
        word_vector = word_vectors.get_vector(i).tolist()
        for i in range(len(centroid)):
            centroid[i] += word_vector[i]
    return centroid
        

In [13]:
centroid = get_query_cent(query)
centroid

[0.348724365234375,
 0.37518310546875,
 0.86669921875,
 0.49951171875,
 -0.53076171875,
 -0.4132080078125,
 0.0966796875,
 -1.15625,
 0.394287109375,
 0.3824462890625,
 0.176513671875,
 -0.546875,
 0.93988037109375,
 -0.066162109375,
 -0.27099609375,
 0.49755859375,
 -0.21502685546875,
 0.501220703125,
 -0.228271484375,
 -0.138671875,
 -0.006072998046875,
 0.5329818725585938,
 -0.105712890625,
 -0.20529937744140625,
 0.487396240234375,
 0.2020263671875,
 -0.8979339599609375,
 0.328369140625,
 0.3327484130859375,
 0.3221435546875,
 -0.055908203125,
 -0.1719970703125,
 -0.482177734375,
 0.003326416015625,
 0.1929779052734375,
 0.23492431640625,
 -0.6435546875,
 -0.2027587890625,
 0.7149658203125,
 0.998046875,
 0.25726318359375,
 -0.087158203125,
 -0.24688720703125,
 0.44244384765625,
 0.10504150390625,
 0.8855743408203125,
 -0.2322998046875,
 0.34722900390625,
 -0.1009521484375,
 0.2477569580078125,
 0.33416748046875,
 0.18841552734375,
 -0.01715087890625,
 -0.191162109375,
 0.175537109

In [14]:
def score_term(t, centroid):
    t=word_vectors.get_vector(t).tolist()
    return np.exp(1-cosine(t,centroid))
    
    

In [78]:
def similar_words( query ): 
    tokens = tokenizer(query)
    words = []
    for i in tokens : 
        similars = model.wv.most_similar(i)
        for j in similars : 
            words.append(j[0])
    return words

In [82]:
test = similar_words(query)

In [21]:
def give_scores_combsum(words, query):
    c= get_query_cent(query)
    tokens = tokenizer(query)
    tokens = list(filter(lambda word: word not in stopwords.words('english'), tokens))
    vectors = [model.wv.get_vector(t) for t in tokens ]
    scores = []
    for j in words : 
        score = 0 
        for i in vectors : 
            score += np.exp(1-cosine(model.wv.get_vector(j),i ))
        scores.append([j,score])
    return scores 

In [20]:
def give_scores_combmax(words, query) :
    c= get_query_cent(query)
    tokens = tokenizer(query)
    tokens = list(filter(lambda word: word not in stopwords.words('english'), tokens))
    vectors = [model.wv.get_vector(t) for t in tokens ]
    scores = []
    for j in words : 
        score = 500 
        for i in vectors : 
            temp = np.exp(1-cosine(model.wv.get_vector(j),i ))
            if temp <score :  ## we should check this one 
                score =tmp
        scores.append([j,score])
    return scores 

In [84]:
tuple_scores  = give_scores_combsum(test,query)

In [87]:
tuple_scores.sort(key=lambda x: x[1], reverse=True)
tuple_scores

[['europe', 10.574700941383536],
 ['europe', 10.574700941383536],
 ['Asia', 10.466483429952872],
 ['Ireland', 10.33892693032024],
 ['asia', 10.330619445129031],
 ['Europe', 10.318848172858925],
 ['Alaska', 10.232309205430203],
 ['alaska', 10.192617546191116],
 ['France', 10.109149089219352],
 ['africa', 10.099518572253261],
 ['hemisphere', 10.088402836754792],
 ['Pennsylvania', 10.085380362425457],
 ['province', 9.993243507887849],
 ['coutry', 9.949219883005412],
 ['te', 9.913510147566932],
 ['tallest', 9.831495153970454],
 ['nation', 9.799727860168906],
 ['teh', 9.79484319496202],
 ['teh', 9.79484319496202],
 ['th', 9.760479242819848],
 ['contry', 9.736034382670447],
 ['largest', 9.718910551469374],
 ['thr', 9.68475150060079],
 ['continent', 9.663200690341604],
 ['hottest', 9.586522345424092],
 ['counrty', 9.582746463885307],
 ['city', 9.566044005544626],
 ['longest', 9.559487465869811],
 ['Judaism', 9.554987388478123],
 ['tha', 9.464411197287829],
 ['tha', 9.464411197287829],
 ['smal

In [22]:
def enrich_queries_CombSum (query,model ): 
    words = similar_words(query)
    tuple_scores = give_scores_combsum(words,query)
    tuple_scores.sort(key=lambda x: x[1], reverse=True)
    return [tuple_scores[i][0] for i in range(3)]


In [23]:
def enrich_queries_combMax (query, model ) : 
    words = similar_words(query)
    tuple_scores = give_scores_combmax(words,query)
    tuple_scores.sort(key=lambda x: x[1], reverse=True)
    return [tuple_scores[i][0] for i in range(3)]


In [107]:
enrich_queries("where can i find a good restaurant ",model )

['buy', 'purchase', 'staff']

## Centroid methods 

In [24]:
def enrich_queries_centroids(query, model) : 
    centroid = get_query_cent(query)
    words =similar_words(query)
    scores = [[i,score_term(i,centroid)] for i in words]
    scores.sort(key=lambda x: x[1], reverse=True)
    return scores 


In [110]:
enrich_queries_centroids("where can i find a good restaurant" ,model )

[['free', 1.8890888209113617],
 ['yourself', 1.7752860895487952],
 ['cant', 1.761182223118025],
 ['pokemon', 1.7237433939515279],
 ['shiny', 1.7065207794480288],
 ['downlaod', 1.6977699764114365],
 ['someone', 1.6919065111990677],
 ['easy', 1.6561933883697564],
 ['somebody', 1.6493030266019262],
 ['cool', 1.6317747509388731],
 ['yo', 1.6313530215056955],
 ['me', 1.6147295712900733],
 ['latias', 1.6008557328798443],
 ['you', 1.5942611751018065],
 ['injected', 1.573143900045362],
 ['should', 1.5633944804854911],
 ['staff', 1.563050571522826],
 ['watch', 1.5574957132063956],
 ['draw', 1.5515222483377504],
 ['account', 1.5488634632035072],
 ['u', 1.5463810928457926],
 ['buy', 1.5453986624286984],
 ['could', 1.543704483200134],
 ['better', 1.5293995114844643],
 ['Can', 1.5245065251315468],
 ['catch', 1.5244930040582116],
 ['joint', 1.5198598154494003],
 ['purchase', 1.511163155234373],
 ['safe', 1.4871524343318514],
 ['healthy', 1.4777219297003894],
 ['might', 1.4652288657833654],
 ['see', 